In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

UTF-8


In [2]:
!pip install nltk datasets transformers[torch]
!pip install tokenizers evaluate rouge_score sentencepiece
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 874.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=912594631f6414ff0d11bdf3eeb4ab4cd0b55143012bde58d0d7cbf964f92f58
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import nltk
import evaluate
import random
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict, load_metric
import datasets
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
from datasets import load_dataset

dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset = dataset.rename_column('Question', 'question')
dataset = dataset.rename_column('Answer', 'answer')
dataset

DatasetDict({
    train: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 16407
    })
})

In [6]:
def reduce_data(raw_datasets, no_of_rows):

  shuffled_dataset = raw_datasets.shuffle(seed=42)
  reduced_data = shuffled_dataset.select(range(no_of_rows))

  return reduced_data

In [7]:
from IPython.display import display, HTML
import datasets
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(dataset['train'])

,qtype,question,answer
0,genetic changes,What are the genetic changes related to tetra-amelia syndrome ?,"Researchers have found a mutation in the WNT3 gene in people with tetra-amelia syndrome from one large family. This gene is part of a family of WNT genes that play critical roles in development before birth. The protein produced from the WNT3 gene is involved in the formation of the limbs and other body systems during embryonic development. Mutations in the WNT3 gene prevent cells from producing functional WNT3 protein, which disrupts normal limb formation and leads to the other serious birth defects associated with tetra-amelia syndrome. In other affected families, the cause of tetra-amelia syndrome has not been determined. Researchers believe that unidentified mutations in WNT3 or other genes involved in limb development are probably responsible for the disorder in these cases."
1,symptoms,What are the symptoms of Autosomal dominant caf au lait spots ?,"What are the signs and symptoms of Autosomal dominant caf au lait spots ? The Human Phenotype Ontology provides the following list of signs and symptoms for Autosomal dominant caf au lait spots . If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to look up the definitions for these medical terms. Signs and Symptoms Approximate number of patients (when available) Cafe-au-lait spot 90% Freckling 7.5% Autosomal dominant inheritance - Lisch nodules - Multiple cafe-au-lait spots - The Human Phenotype Ontology (HPO) has collected information on how often a sign or symptom occurs in a condition. Much of this information comes from Orphanet, a European rare disease database. The frequency of a sign or symptom is usually listed as a rough estimate of the percentage of patients who have that feature. The frequency may also be listed as a fraction. The first number of the fraction is how many people had the symptom, and the second number is the total number of people who were examined in one study. For example, a frequency of 25/25 means that in a study of 25 people all patients were found to have that symptom. Because these frequencies are based on a specific study, the fractions may be different if another group of patients are examined. Sometimes, no information on frequency is available. In these cases, the sign or symptom may be rare or common."
2,information,What is (are) Earthquakes ?,"An earthquake happens when two blocks of the earth suddenly slip past one another. Earthquakes strike suddenly, violently, and without warning at any time of the day or night. If an earthquake occurs in a populated area, it may cause property damage, injuries, and even deaths. If you live in a coastal area, there is the possibility of a tsunami. Damage from earthquakes can also lead to floods or fires. Although there are no guarantees of safety during an earthquake, you can take actions to protect yourself. You should have a disaster plan. Being prepared can help reduce fear, anxiety, and losses. If you do experience a disaster, it is normal to feel stressed. You may need help in finding ways to cope. Federal Emergency Management Agency"
3,symptoms,What are the symptoms of Sideroblastic anemia pyridoxine-refractory autosomal recessive ?,"What are the signs and symptoms of Sideroblastic anemia pyridoxine-refractory autosomal recessive? The symptoms of sideroblastic anemia are the same as for any anemia and iron overload. These may include fatigue, weakness, palpitations, shortness of breath, headaches, irritability, and chest pain. Physical findings may include pallor, tachycardia, hepatosplenomegaly, S3 gallop, jugular vein distension, and rales. Some people with sideroblastic anemia develop diabetes or abnormal glucose tolerance which may or may not be related to the degree of iron overload. The most dangerous complication of iron overload are heart arrhythmias and heart failure,

In [9]:
dataset_split = dataset['train'].train_test_split(test_size=0.2)
val_test_split = dataset_split['test'].train_test_split(test_size=0.5)
# print(dataset_split)
# print(val_test_split)

final_dataset_dict = DatasetDict({
    # 'train': reduce_data(dataset_split['train'], 131),
    # 'validation': reduce_data(val_test_split['train'], 16),
    # 'test': reduce_data(val_test_split['test'], 16)
    'train': dataset_split['train'],
    'validation': val_test_split['train'],
    'test': val_test_split['test'],
})
final_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 13125
    })
    validation: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 1641
    })
    test: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 1641
    })
})

In [12]:
model_checkpoint = "google/flan-t5-base"
# model_checkpoint = "luqh/ClinicalT5-base"

In [13]:
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model = model)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# model_checkpoint = "dmis-lab/biobert-base-cased-v1.1-squad"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model = model)

In [15]:
max_input_length = 128
max_target_length = 256

prefix = "answer the question: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # edit: counting the avg length of input
    # print(model_inputs)
    # lens = [len(pred) for pred in model_inputs]
    # print("mean question length" , np.mean(lens))
    # print(examples["question"][0].split(' '))
    # lens_ = [len(pred.split(' ')) for pred in examples["question"]]
    # print("mean question length" , np.mean(lens_))

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["answer"], max_length=max_target_length, truncation=True)
    # labels = tokenizer(labels, max_length=max_target_length, truncation=True)

    # print(labels)
    # lens = [len(pred) for pred in labels]
    # print("mean answer length" , np.mean(lens))

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
tokenizer.pad_token_id

0

In [17]:
tokenized_datasets = final_dataset_dict.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1641 [00:00<?, ? examples/s]

Map:   0%|          | 0/1641 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['qtype', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13125
    })
    validation: Dataset({
        features: ['qtype', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1641
    })
    test: Dataset({
        features: ['qtype', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1641
    })
})

In [18]:
from evaluate import load
rouge_metric = load('rouge')

In [28]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}


    # Add mean generated length
    # print(predictions)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    print(result)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
batch_size = 8
no_of_train_epochs = 3
model_name = model_checkpoint.split("/")[-1]
learning_rate = 3e-4
per_device_eval_batch = 4

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    report_to=None,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=per_device_eval_batch,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=no_of_train_epochs,
    predict_with_generate=True,
    # push_to_hub=True,
)

In [39]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
torch.cuda.empty_cache()

In [41]:
trainer.train()
# NOTE: Rouge Scores are multiplied by 100

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.769200,1.522432,16.518800,10.621500,15.195500,15.888000,18.933600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 16.51882129312801, 'rouge2': 10.621475286114665, 'rougeL': 15.19545604054168, 'rougeLsum': 15.88803523040528, 'gen_len': 18.933577087141988}


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.769200,1.522432,16.518800,10.621500,15.195500,15.888000,18.933600
2,1.576700,1.440207,16.938100,11.132400,15.712600,16.375700,18.912900
3,1.463600,1.417418,16.835900,11.092900,15.610800,16.263000,18.920200


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 16.938054502711577, 'rouge2': 11.132362748591577, 'rougeL': 15.712602181353791, 'rougeLsum': 16.375743958129014, 'gen_len': 18.91285801340646}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 16.835927298064426, 'rouge2': 11.092923298479876, 'rougeL': 15.610828513761824, 'rougeLsum': 16.26300727787301, 'gen_len': 18.920170627666057}


TrainOutput(global_step=4923, training_loss=1.624563852171091, metrics={'train_runtime': 4605.2523, 'train_samples_per_second': 8.55, 'train_steps_per_second': 1.069, 'total_flos': 1499332394686464.0, 'train_loss': 1.624563852171091, 'epoch': 3.0})

In [ ]:
import os
from google.colab import drive

drive_path = "/content/drive/MyDrive/Augnito/models/"

if os.path.exists(drive_path) == False:
  drive.mount('/content/drive')

In [43]:
model_checkpoint

'google/flan-t5-base'

In [44]:
import os

def save_model(model, tokenizer, model_path):
  if not os.path.exists(model_path):
    os.makedirs(model_path)
  print("Directory created for new checkpt to save")

  model.save_pretrained(model_path)
  tokenizer.save_pretrained(model_path)

model_name = model_checkpoint.split('/')[-1]
model_path = "/content/drive/MyDrive/Augnito/models/" + f"{model_name}-{batch_size}-{no_of_train_epochs}"
save_model(model, tokenizer, model_path)

Directory created for new checkpt to save


In [45]:
prefix

'answer the question: '

In [69]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [116]:
def __generate(query):
    inputs = tokenizer(prefix + query, return_tensors="pt").to(device).input_ids
    # print(inputs)
    outputs = model.generate(inputs, num_beams=4)
    # print(outputs)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(answer)

    return answer

In [117]:
for i in range(5):
  query = tokenized_datasets['validation']['question'][i]
  print("query: ", query)
  print("generated ans: ", __generate(query))
  print()
  print()

query:  What causes Hereditary leiomyomatosis and renal cell cancer ?
generated ans:  What causes hereditary leiomyomatosis and renal cell cancer? Hereditary


query:  What is (are) Meningioma ?
generated ans:  Key Points - Meningioma is a disease in which malignant


query:  what research (or clinical trials) is being done for Hypertonia ?
generated ans:  The National Institute of Neurological Disorders and Stroke (NINDS) and other


query:  Who is at risk for Peripheral Arterial Disease (P.A.D.)? ?
generated ans:  The risk factors for P.A.D. include the following: - Age -


query:  Do you have information about Antibiotics
generated ans:  Summary : Antibiotics are a group of medicines used to treat diseases and conditions.


